In [5]:
import numpy as np
from collections import Counter


class MyDecisionTreeClassifier:

    def __init__(self, max_depth=None, max_features=None, min_leaf_samples=None):
        self.max_depth = max_depth
        self.max_features = max_features
        self.min_leaf_samples = min_leaf_samples
        self._node = {
                        'left': None,
                        'right': None,
                        'feature': None,
                        'threshold': None,
                        'depth': 0,
                        'classes_proba': None
                    }
        self.tree = None  # словарь в котором будет храниться построенное дерево
        self.classes = None  # список меток классов

    def fit(self, X, y):
        self.classes = np.unique(y)  
        self.tree = {'root': self._node.copy()}  # создаём первую узел в дереве
        self._build_tree(self.tree['root'], X, y)  # запускаем рекурсивную функцию для построения дерева
        return self

    def predict_proba(self, X):
        proba_preds = []
        for x in X:
            preds_for_x = self._get_predict(self.tree['root'], x)  # рекурсивно ищем лист в дереве соответствующий объекту
            proba_preds.append(preds_for_x)
        return np.array(proba_preds)

    def predict(self, X):
        proba_preds = self.predict_proba(X)
        preds = proba_preds.argmax(axis=1).reshape(-1, 1)
        return preds

    def get_best_split(self, X, y):

        best_Q = None
        best_j = None
        best_t = None
        best_left_ids = None
        best_right_ids = None
        
        full_gini = self.gini(y)
        features = X.shape[1]
        for j in range(features):
            split_values = self.find_splits(X, j)
            for t in split_values:
                left_mask = X[:, j] <= t
                right_mask = X[:, j] > t
                left_y, right_y = y[left_mask], y[right_mask]
                Q_cur = self.calc_Q(left_y, right_y, full_gini)
                if best_Q is None or Q_cur > best_Q:
                    best_Q = Q_cur
                    best_j = j
                    best_t = t
                    best_left_ids = left_mask
                    best_right_ids = right_mask

        return best_j, best_t, best_left_ids, best_right_ids

    def calc_Q(self, y_left, y_right, full_gini):
        n_l = len(y_left)
        n_r = len(y_right)
        n = n_l + n_r
        return full_gini - (n_l*self.gini(y_left) + n_r*self.gini(y_right))/n

    def gini(self, y):
        n = len(y)
        if n < 2:
            return 0
        y=np.array(y).reshape(1, -1).squeeze()
        counter = Counter(y)
        arr = []
        for c in counter:
            arr.append((counter[c]/n)**2)
        arr = np.array(arr)
        return 1 - np.sum(arr)
    
    def find_splits(self, X, column):
        X_unique = np.unique(X[:, column])
        split_values = np.empty(X_unique.shape[0] - 1)
        for i in range(1, X_unique.shape[0]):
            average = (X_unique[i - 1] + X_unique[i]) / 2
            split_values[i - 1] = average
        return split_values

    def _build_tree(self, curr_node, X, y):

        if curr_node['depth'] == self.max_depth:  # выход из рекурсии если построили до максимальной глубины
            # сохраняем предсказания листьев дерева перед выходом из рекурсии
            curr_node['classes_proba'] = {c: (y == c).mean() for c in self.classes}
            return

        if len(np.unique(y)) == 1:  # выход из рекурсии значения если "y" одинковы для все объектов
            curr_node['classes_proba'] = {c: (y == c).mean() for c in self.classes}
            return

        j, t, left_ids, right_ids = self.get_best_split(X, y)  # нахождение лучшего разбиения

        curr_node['feature'] = j  # признак по которому производится разбиение в текущем узле
        curr_node['threshold'] = t  # порог по которому производится разбиение в текущем узле

        left = self._node.copy()  # создаём узел для левого поддерева
        right = self._node.copy()  # создаём узел для правого поддерева

        left['depth'] = curr_node['depth'] + 1  # увеличиваем значение глубины в узлах поддеревьев
        right['depth'] = curr_node['depth'] + 1

        curr_node['left'] = left
        curr_node['right'] = right

        self._build_tree(left, X[left_ids], y[left_ids])  # продолжаем построение дерева
        self._build_tree(right, X[right_ids], y[right_ids])

    def _get_predict(self, node, x):
        if node['threshold'] is None:  # если в узле нет порога, значит это лист, выходим из рекурсии
            return [node['classes_proba'][c] for c in self.classes]

        if x[node['feature']] <= node['threshold']:  # уходим в правое или левое поддерево в зависимости от порога и признака
            return self._get_predict(node['left'], x)
        else:
            return self._get_predict(node['right'], x)


def read_matrix(n, dtype=float):
    matrix = np.array([list(map(dtype, input().split())) for _ in range(n)])
    return matrix

def read_input_matriсes(n, m, k):
    X_train, y_train, X_test = read_matrix(n), read_matrix(n), read_matrix(k)
    return X_train, y_train, X_test

def print_matrix(matrix):
    for row in matrix:
        print(' '.join(map(str, row)))

def solution():
    n, m, k = 3, 2, 2
    X_train = np.array([[0.0, 1.0],
                        [0.0, 2.0], 
                        [0.0, 3.0]])
    y_train = np.array([[1.], [2.], [3.]]) 
    X_test = np.array([[0.0, 1.0], [0.0, 2.0]])

    clf = MyDecisionTreeClassifier()
    clf.fit(X_train, y_train)
    preds = clf.predict(X_test)
    proba_preds = clf.predict_proba(X_test).round(4)

    print_matrix(preds)
    print_matrix(proba_preds)


solution()

0
1
1.0 0.0 0.0
0.0 1.0 0.0


In [ ]:
def entropy(x):
    n = len(x)
    p1 = np.sum(x) / n
    p2 = 1 - p1
    if n == 0:
        return 0
    if p2 and p1:
        return -(p1*np.log(p1) + p2*np.log(p2))
    return 0
entropy([0, 0, 0, 0])

In [1]:
import numpy as np

In [75]:
a=np.array([[1], [2], [3]]).reshape(1, -1).squeeze()

In [76]:
np.sum(a, axis=0)

6

In [77]:
a

array([1, 2, 3])

In [79]:
c = Counter(a)
c

Counter({1: 1, 2: 1, 3: 1})

In [94]:
a=[[1.0], [2.0], [3.0]]
a[0][:]

[1.0]

In [31]:
def print_matrix(matrix):
    for row in matrix:
        print(' '.join(map(str, row)))

In [33]:
print_matrix(a)

1
2
3


In [34]:
import numpy as np
from collections import Counter


class MyDecisionTreeClassifier:

    def __init__(self, max_depth=None, max_features=None, min_leaf_samples=None):
        self.max_depth = max_depth
        self.max_features = max_features
        self.min_leaf_samples = min_leaf_samples
        self._node = {
                        'left': None,
                        'right': None,
                        'feature': None,
                        'threshold': None,
                        'depth': 0,
                        'classes_proba': None
                    }
        self.tree = None  # словарь в котором будет храниться построенное дерево
        self.classes = None  # список меток классов

    def fit(self, X, y):
        self.classes = np.unique(y)  
        self.tree = {'root': self._node.copy()}  # создаём первую узел в дереве
        self._build_tree(self.tree['root'], X, y)  # запускаем рекурсивную функцию для построения дерева
        return self

    def predict_proba(self, X):
        proba_preds = []
        for x in X:
            preds_for_x = self._get_predict(self.tree['root'], x)  # рекурсивно ищем лист в дереве соответствующий объекту
            proba_preds.append(preds_for_x)
        return np.array(proba_preds)

    def predict(self, X):
        proba_preds = self.predict_proba(X)
        preds = proba_preds.argmax(axis=1).reshape(-1, 1)
        return preds

    def get_best_split(self, X, y):

        best_Q = -1
        best_j = -1
        best_t = -1
        best_left_ids = []
        best_right_ids = []
        features = X.shape[1]
        for j in range(features):
            t_min = min(X[:, j])
            for i, t in enumerate(X[:, j]):
                l_arr = [y_ for x, y_ in zip(X[:, j], y) if x < t]
                r_arr = [y_ for x, y_ in zip(X[:, j], y) if x >= t]
                print(r_arr)
                Q_cur = self.calc_Q(y, l_arr, r_arr)
                if Q_cur > best_Q:
                    best_Q = Q_cur
                    best_j = j
                    best_t = (t + X[i - 1][j])/2 if t > t_min else t
                    best_left_ids = np.array([1 if y_ in l_arr else 0 for y_ in y])
                    best_right_ids = np.array([1 if y_ in r_arr else 0 for y_ in y])

        return best_j, best_t, best_left_ids, best_right_ids

    def calc_Q(self, y, y_left, y_right):
        n_l = len(y_left)
        n_r = len(y_right)
        n = len(y)
        return self.gini(y) - (n_l*self.gini(y_left) + n_r*self.gini(y_right))/n

    def gini(self, y):
        print(y)
        n = len(y)
        #p1 = np.sum()
        return 1 

    def _build_tree(self, curr_node, X, y):

        if curr_node['depth'] == self.max_depth:  # выход из рекурсии если построили до максимальной глубины
            curr_node['classes_proba'] = {c: (y == c).mean() for c in self.classes}  # сохраняем предсказания листьев дерева перед выходом из рекурсии
            return

        if len(np.unique(y)) == 1:  # выход из рекурсии значения если "y" одинковы для все объектов
            curr_node['classes_proba'] = {c: (y == c).mean() for c in self.classes}
            return

        j, t, left_ids, right_ids = self.get_best_split(X, y)  # нахождение лучшего разбиения

        curr_node['feature'] = j  # признак по которому производится разбиение в текущем узле
        curr_node['threshold'] = t  # порог по которому производится разбиение в текущем узле

        left = self._node.copy()  # создаём узел для левого поддерева
        right = self._node.copy()  # создаём узел для правого поддерева

        left['depth'] = curr_node['depth'] + 1  # увеличиваем значение глубины в узлах поддеревьев
        right['depth'] = curr_node['depth'] + 1

        curr_node['left'] = left
        curr_node['right'] = right

        self._build_tree(left, X[left_ids], y[left_ids])  # продолжаем построение дерева
        self._build_tree(right, X[right_ids], y[right_ids])

    def _get_predict(self, node, x):
        if node['threshold'] is None:  # если в узле нет порога, значит это лист, выходим из рекурсии
            return [node['classes_proba'][c] for c in self.classes]

        if x[node['feature']] <= node['threshold']:  # уходим в правое или левое поддерево в зависимости от порога и признака
            return self._get_predict(node['left'], x)
        else:
            return self._get_predict(node['right'], x)


def read_matrix(n, dtype=float):
    matrix = np.array([list(map(dtype, input().split())) for _ in range(n)])
    return matrix

def read_input_matriсes(n, m, k):
    X_train, y_train, X_test = read_matrix(n), read_matrix(n), read_matrix(k)
    return X_train, y_train, X_test

def print_matrix(matrix):
    for row in matrix:
        print(' '.join(map(str, row)))

def solution():
    n, m, k = map(int, input().split())
    X_train, y_train, X_test = read_input_matriсes(n, m, k)

    clf = MyDecisionTreeClassifier()
    clf.fit(X_train, y_train)
    preds = clf.predict(X_test)
    proba_preds = clf.predict_proba(X_test).round(4)

    print_matrix(preds)
    print_matrix(proba_preds)


solution()

3 2 2
0 1
0 2
0 3
1
2
3
0 1
0 2
[array([1.]), array([2.]), array([3.])]
[[1.]
 [2.]
 [3.]]
[]
[array([1.]), array([2.]), array([3.])]
[array([1.]), array([2.]), array([3.])]
[[1.]
 [2.]
 [3.]]
[]
[array([1.]), array([2.]), array([3.])]
[array([1.]), array([2.]), array([3.])]
[[1.]
 [2.]
 [3.]]
[]
[array([1.]), array([2.]), array([3.])]
[array([1.]), array([2.]), array([3.])]
[[1.]
 [2.]
 [3.]]
[]
[array([1.]), array([2.]), array([3.])]
[array([2.]), array([3.])]
[[1.]
 [2.]
 [3.]]
[array([1.])]
[array([2.]), array([3.])]
[array([3.])]
[[1.]
 [2.]
 [3.]]
[array([1.]), array([2.])]
[array([3.])]
0
0
1.0 0.0 0.0
1.0 0.0 0.0


In [ ]:
3 2 2
0.0 1.0 
0.0 2.0 
0.0 3.0 
1
2
3
0.0 1.0
0.0 2.0